<a href="https://colab.research.google.com/github/rtedwards/nn-zero-to-hero/blob/main/notebooks/4_makemore_backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Back Propagation
- [Yes you should understand BackProp](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b)
- Need to understand back prop to be able to debug

In [2]:
import random

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm

In [3]:
words = open("names.txt", "r").read().splitlines()

In [4]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


## I. Build Dataset

In [5]:
def build_dataset(words, block_size = 3):
    """
    - block_size - context length: how many characters fo we take to predict the next one?
    """
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)

            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)

    print(X.shape, Y.shape)
    return X, Y

## II. Data set Splits

- Training (80%) - used to optimize the model parameters
- Validation (10%) - used to optimize the model hyperparameters(number of neurons, size of hidden layer, size of embedding, strength of regularization)
- Test (10%) - used to evaluate model as it generalizes to unseen data

In [6]:
# train / validation / test splits
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
block_size = 3 # number of previous characters

X_train, Y_train = build_dataset(words[:n1], block_size)
X_val, Y_val = build_dataset(words[n1:n2], block_size)
X_test, Y_test = build_dataset(words[n2:], block_size)

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [10]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
# embedding layer for characters
C  = torch.randn((vocab_size, n_embd),            generator=g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN

# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1

# BatchNorm parameters
bn_gain = torch.randn((1, n_hidden))*0.1 + 1.0
bn_bias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True


4137


In [11]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience

# construct a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
Xb, Yb = X_train[ix], Y_train[ix] # batch X,Y

In [12]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation

# BatchNorm layer
bn_mean_i = 1/n*hprebn.sum(0, keepdim=True)
bn_diff = hprebn - bn_mean_i
bn_diff2 = bn_diff**2
bn_var = 1/(n-1)*(bn_diff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bn_var_inv = (bn_var + 1e-5)**-0.5
bn_raw = bn_diff * bn_var_inv
hpreact = bn_gain * bn_raw + bn_bias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bn_raw,
         bn_var_inv, bn_var, bn_diff2, bn_diff, hprebn, bn_mean_i,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3506, grad_fn=<NegBackward0>)

In [26]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

d_logprobs = torch.zeros_like(logprobs) #1 / probs[range(n)].log() # 1 / (x ln a)
d_logprobs[range(n), Yb] = -1.0 / n # d/dxi of -(x1 + x2 + ... + xn) / n
d_probs = 1 / probs * d_logprobs    # (local_derivative * chained_derivatives)
d_counts_sum_inv = (counts * d_probs).sum(1, keepdim=True) # (local_derivative * chained_derivatives) then sum across columns
d_counts = counts_sum_inv * d_probs
d_counts_sum = -(counts_sum)**-2 * d_counts_sum_inv
d_counts += torch.ones_like(counts) * d_counts_sum # (add the 2 branches that rely on counts)
d_norm_logits = norm_logits.exp() * d_counts
d_logits = d_norm_logits.clone()
d_logit_maxes = (-d_norm_logits).sum(1, keepdim=True)
d_logits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * d_logit_maxes
d_h = None
d_W2 = None
d_b2 = None
d_hpreact = None
d_bn_gain = None
d_bn_bias = None
d_bn_raw = None
d_bn_var_inv = None
d_bn_var = None
d_bn_diff2 = None
d_bn_diff = None
d_bn_mean_i = None
d_hprebn = None
d_embcat = None
d_W1 = None
d_b1 = None
d_emb = None
d_C = None

cmp('logprobs', d_logprobs, logprobs)
cmp('probs', d_probs, probs)
cmp('counts_sum_inv', d_counts_sum_inv, counts_sum_inv)
cmp('counts_sum', d_counts_sum, counts_sum)
cmp('counts', d_counts, counts)
cmp('norm_logits', d_norm_logits, norm_logits)
cmp('logit_maxes', d_logit_maxes, logit_maxes)
cmp('logits', d_logits, logits)
# cmp('h', d_h, h)
# cmp('W2', d_W2, W2)
# cmp('b2', d_b2, b2)
# cmp('hpreact', d_hpreact, hpreact)
# cmp('bngain', d_bn_gain, bn_gain)
# cmp('bnbias', d_bn_bias, bn_bias)
# cmp('bnraw', d_bn_raw, bn_raw)
# cmp('bnvar_inv', d_bn_var_inv, bn_var_inv)
# cmp('bnvar', d_bn_var, bn_var)
# cmp('bndiff2', d_bn_diff2, bn_diff2)
# cmp('bndiff', d_bn_diff, bn_diff)
# cmp('bnmeani', d_bn_mean_i, bn_mean_i)
# cmp('hprebn', d_hprebn, hprebn)
# cmp('embcat', d_embcat, embcat)
# cmp('W1', d_W1, W1)
# cmp('b1', d_b1, b1)
# cmp('emb', d_emb, emb)
# cmp('C', d_C, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0


In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass
# --------
# Your Code Here
# --------

cmp('logits', d_logits, logits) # I can only get approximate to be true, my maxdiff is 6e-9